In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the data
df_train = pd.read_csv("train.csv", index_col='id')
df_test = pd.read_csv("test.csv", index_col='id')
submission = pd.read_csv("sample_submission.csv", index_col='id')
# Separate target from predictors
df_target = df_train.pop('target')

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(df_train, df_target, train_size=0.8, test_size=0.2, random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitary)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
'''
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
'''

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        # ('cat', categorical_transformer, categorical_cols)
    ])

In [14]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the model
xgbmodel = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
xgbmodel.fit(X_train, y_train)

# Get predictions
prediction = xgbmodel.predict(X_valid)

# Calculate MAE
mae = mean_absolute_error(prediction, y_valid)
print("Mean Absolute Error:" , mae)

ModuleNotFoundError: No module named 'xgboost'

In [10]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
def RMSE(y_valid, preds):
    return np.sqrt(mean_squared_error(y_valid, preds))
                   
print(RMSE(y_valid, preds))

preds_test = my_pipeline.predict(df_test)

0.7089659342379675


In [11]:
submission['target'] = preds_test
submission.to_csv('Tabular_Jan2021_submission.csv')